In [1]:
from pprint import pprint
from dataclasses import asdict
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from martignac.workflows.solute_generation import SoluteGenFlow
from martignac.utils.martini_flow_projects import MartiniFlowProject
from martignac.nomad.uploads import get_upload_by_id, get_all_my_uploads
from martignac.nomad.datasets import get_dataset_by_id
from martignac.nomad.users import get_user_by_id
from martignac.nomad.entries import (
    query_entries, 
    get_entry_by_id, 
    get_entries_of_upload, 
    find_entries_corresponding_to_job, 
    download_raw_data_of_job,
    get_entries_of_my_uploads,
    get_entries_in_database
)

24-03-19 09:55:47 - MDAnalysis.coordinates.AMBER - WARNING - netCDF4 is not available. Writing AMBER ncdf files will be slow.


In [2]:
job = list(SoluteGenFlow.get_project(path=SoluteGenFlow.workspace_path).find_jobs())[0]
job

Job(project=SoluteGenFlow('/Users/bereau/work/projects/martignac/workspaces/solute_generation'), statepoint={'type': 'solute', 'solute_name': 'P6'})

In [3]:
job.id

'bc3c7ca12d4e7b0fa984e8cbb813acbf'

In [22]:
job.document.update?

Signature: job.document.update(other=None, **kwargs)
Docstring:
D.update([E, ]**F) -> None.  Update D from mapping/iterable E and F.
If E present and has a .keys() method, does:     for k in E: D[k] = E[k]
If E present and lacks .keys() method, does:     for (k, v) in E: D[k] = v
In either case, this is followed by: for k, v in F.items(): D[k] = v
File:      ~/work/projects/martignac/.venv/lib/python3.9/site-packages/synced_collections-1.0.0-py3.9.egg/synced_collections/data_types/synced_dict.py
Type:      method

In [6]:
entries = get_entries_in_database()
pd.json_normalize(asdict(e) for e in entries)

24-03-18 15:24:48 - martignac.nomad.utils - INFO - Sending post request @ https://nomad-lab.eu/prod/v1/test/api/v1/entries/query
24-03-18 15:24:48 - martignac.nomad.entries - INFO - retrieving entry IWfLp8VCyT7z9t3BtVy21Q5WZSRW on test server
24-03-18 15:24:48 - martignac.nomad.utils - INFO - Sending get request @ https://nomad-lab.eu/prod/v1/test/api/v1/entries/IWfLp8VCyT7z9t3BtVy21Q5WZSRW
24-03-18 15:24:48 - martignac.nomad.users - INFO - retrieving user 30d3a108-d2cc-45ec-9ddb-0c1dc6a2c99b on prod server
24-03-18 15:24:48 - martignac.nomad.utils - INFO - Sending get request @ https://nomad-lab.eu/prod/v1/test/api/v1/users/30d3a108-d2cc-45ec-9ddb-0c1dc6a2c99b


,entry_id,upload_id,references,origin,quantities,datasets,n_quantities,nomad_version,upload_create_time,nomad_commit,...,main_author.first_name,main_author.last_name,main_author.username,main_author.affiliation,main_author.affiliation_address,main_author.email,main_author.is_oasis_admin,main_author.is_admin,main_author.repo_user_id,main_author.created
0,IWfLp8VCyT7z9t3BtVy21Q5WZSRW,Hbwh9S0-RLehQIh90nfhAQ,[],Tristan Bereau,"[, metadata, metadata.coauthor_groups, metadat...","[{'dataset_id': 'HJdEI1q4SV-c5Di43BTT_Q', 'dat...",0,1.2.2.dev365+g0c980916a,2024-02-14 09:43:58.640000+00:00,,...,Tristan,Bereau,tbereau,Heidelberg University,"Heidelberg, Germany",None,None,None,None,None


In [7]:
entries = get_entries_of_my_uploads()
pd.json_normalize([{**asdict(e), "nomad_url": e.nomad_gui_url} for e in entries])

,entry_id,upload_id,references,origin,quantities,datasets,n_quantities,nomad_version,upload_create_time,nomad_commit,...,main_author.last_name,main_author.username,main_author.affiliation,main_author.affiliation_address,main_author.email,main_author.is_oasis_admin,main_author.is_admin,main_author.repo_user_id,main_author.created,results.properties.geometry_optimization.convergence_tolerance_force_maximum
0,IWfLp8VCyT7z9t3BtVy21Q5WZSRW,Hbwh9S0-RLehQIh90nfhAQ,[],Tristan Bereau,"[, metadata, metadata.coauthor_groups, metadat...","[{'dataset_id': 'HJdEI1q4SV-c5Di43BTT_Q', 'dat...",0,1.2.2.dev365+g0c980916a,2024-02-14 09:43:58.640000+00:00,,...,Bereau,tbereau,Heidelberg University,"Heidelberg, Germany",None,None,None,None,None,NaN
1,WzTeM_gAU_3LyARE4isJHUk-Hykw,Rh_T1ALjQMGW062Lwn06kA,[],Tristan Bereau,"[, metadata, metadata.coauthor_groups, metadat...","[{'dataset_id': 'HJdEI1q4SV-c5Di43BTT_Q', 'dat...",52,1.2.2.dev365+g0c980916a,2024-03-15 09:38:00.440000+00:00,,...,Bereau,tbereau,Heidelberg University,"Heidelberg, Germany",None,None,None,None,None,NaN
2,c5_NBBaMfeRFZZruEbYRRq-lK096,Rh_T1ALjQMGW062Lwn06kA,[],Tristan Bereau,"[, metadata, metadata.coauthor_groups, metadat...","[{'dataset_id': 'HJdEI1q4SV-c5Di43BTT_Q', 'dat...",244,1.2.2.dev365+g0c980916a,2024-03-15 09:38:00.440000+00:00,,...,Bereau,tbereau,Heidelberg University,"Heidelberg, Germany",None,None,None,None,None,6.022141e+36
